In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
27,application_1579214001909_0028,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
fulltrain = ss.read.parquet("s3://usf.msds.stillbikeshare/full_train")
fulltest = ss.read.parquet("s3://usf.msds.stillbikeshare/full_test")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
selectedtrain = ss.read.parquet("s3://usf.msds.stillbikeshare/selected_train")
selectedtest = ss.read.parquet("s3://usf.msds.stillbikeshare/selected_test")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
strainclean = selectedtrain.filter("label <= 43200")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
stestclean = selectedtrain.filter("label <= 43200")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Regularized Linear Regression - Selected Features

In [7]:

# Load training data
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# # Fit the model
# lrModel = lr.fit(training)

# # Print the weights and intercept for linear regression
# print("Weights: " + str(lrModel.weights))
# print("Intercept: " + str(lrModel.intercept))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
evaluator = RegressionEvaluator()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
va = VectorAssembler(outputCol="features", inputCols=selectedtrain.columns[:-1])
selectedtrainingset = va.transform(selectedtrain).select("features", "label").cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
selectedtestingset = va.transform(selectedtest).select("features", "label").cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Cross-validation - Selected Features

In [11]:
cv = CrossValidator().setEstimator(lr).setEvaluator(evaluator).setNumFolds(5)
#ParamGridBuilder() – combinations of parameters and their values.
paramGrid = ParamGridBuilder().addGrid(lr.maxIter, [1000]).addGrid(lr.regParam, [0.01, 0.1, 0.3]).addGrid(lr.elasticNetParam, [0.6, 0.8]).build()
#setEstimatorParamMaps() takes ParamGridBuilder().
cv.setEstimatorParamMaps(paramGrid)
cvmodel = cv.fit(selectedtrainingset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
RegressionEvaluator().evaluate(cvmodel.bestModel.transform(selectedtestingset))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4412.567394750101

# Unregularized Linear Regression - Selected Features

In [13]:
lr2 = LinearRegression(maxIter=10, regParam=0.3)
evaluator2 = RegressionEvaluator()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# va2 = VectorAssembler(outputCol="features", inputCols=selectedtrain.columns[:-1])
# fulltrainingset = va2.transform(fulltrain).select("features", "label").cache()
# fulltestingset = va2.transform(fulltest).select("features", "label").cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Cross-validation - Selected Features

In [15]:
cv2 = CrossValidator().setEstimator(lr2).setEvaluator(evaluator2).setNumFolds(5)
#ParamGridBuilder() – combinations of parameters and their values.
paramGrid = ParamGridBuilder().addGrid(lr2.maxIter, [1000]).addGrid(lr2.regParam, [0.01, 0.1, 0.3]).build()
#setEstimatorParamMaps() takes ParamGridBuilder().
cv2.setEstimatorParamMaps(paramGrid)
cvmodel2 = cv2.fit(selectedtrainingset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
RegressionEvaluator().evaluate(cvmodel2.bestModel.transform(selectedtestingset))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4399.719171526276

# Regularized LR with Outliners Removed

In [17]:

# Load training data
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# # Fit the model
# lrModel = lr.fit(training)

# # Print the weights and intercept for linear regression
# print("Weights: " + str(lrModel.weights))
# print("Intercept: " + str(lrModel.intercept))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
evaluator = RegressionEvaluator()
va = VectorAssembler(outputCol="features", inputCols=selectedtrain.columns[:-1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
notrainingset = va.transform(strainclean).select("features", "label").cache()
notestingset = va.transform(strainclean).select("features", "label").cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
cv = CrossValidator().setEstimator(lr).setEvaluator(evaluator).setNumFolds(5)
#ParamGridBuilder() – combinations of parameters and their values.
paramGrid = ParamGridBuilder().addGrid(lr.maxIter, [10, 1000]).addGrid(lr.regParam, [0.01, 0.1, 0.3]).addGrid(lr.elasticNetParam, [0.9, 0.6, 0.8]).build()
#setEstimatorParamMaps() takes ParamGridBuilder().
cv.setEstimatorParamMaps(paramGrid)
cvmodel = cv.fit(notrainingset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
RegressionEvaluator().evaluate(cvmodel.bestModel.transform(notestingset))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1161.2100715875486

# Unregularized LR with Outliners Removed

In [28]:
lr2 = LinearRegression(maxIter=10, regParam=0.3)
evaluator2 = RegressionEvaluator()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
cv2 = CrossValidator().setEstimator(lr2).setEvaluator(evaluator2).setNumFolds(5)
#ParamGridBuilder() – combinations of parameters and their values.
paramGrid = ParamGridBuilder().addGrid(lr2.maxIter, [10, 1000]).addGrid(lr2.regParam, [0.01, 0.1, 0.3]).build()
#setEstimatorParamMaps() takes ParamGridBuilder().
cv2.setEstimatorParamMaps(paramGrid)
cvmodel2 = cv2.fit(notrainingset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
RegressionEvaluator().evaluate(cvmodel2.bestModel.transform(notestingset))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1160.9540457824405